In [3]:
TaskObject = ""

In [97]:
from notebookutils import mssparkutils
import re
import json
import pandas as pd
from datetime import date
from datetime import datetime
from pyspark.sql import Row
from pyspark.sql.types import *
from pyspark.sql.functions import *
from delta.tables import *
#TaskObject = "{\"TaskInstanceId\":10,\"TaskMasterId\":2,\"TaskStatus\":\"InProgress\",\"TaskType\":\"Execute Synapse Notebook\",\"Enabled\":1,\"ExecutionUid\":\"29283663-9778-4c37-9100-1307f98ac2f4\",\"NumberOfRetries\":2,\"DegreeOfCopyParallelism\":1,\"KeyVaultBaseUrl\":\"https://ark-stg-kv-ads-ikjn.vault.azure.net/\",\"ScheduleMasterId\":\"-4\",\"TaskGroupConcurrency\":\"10\",\"TaskGroupPriority\":0,\"TaskExecutionType\":\"ADF\",\"ExecutionEngine\":{\"EngineId\":-2,\"EngineName\":\"arkstgsynwadsikjn\",\"SystemType\":\"Synapse\",\"ResourceGroup\":\"LockboxDev01\",\"SubscriptionId\":\"687fe1ae-a520-4f86-b921-a80664c40f9b\",\"ADFPipeline\":\"GPL_SparkNotebookExecution_Azure\",\"EngineJson\":\"{\\r\\n            \\\"endpoint\\\": \\\"https://arkstgsynwadsikjn.dev.azuresynapse.net\\\", \\\"DeltaProcessingNotebook\\\": \\\"DeltaProcessingNotebook\\\", \\\"PurviewAccountName\\\": \\\"arkstgpurads\\\", \\\"DefaultSparkPoolName\\\":\\\"Dummy\\\"\\r\\n        }\",\"TaskDatafactoryIR\":\"Azure\",\"JsonProperties\":{\"endpoint\":\"https://arkstgsynwadsikjn.dev.azuresynapse.net\",\"DeltaProcessingNotebook\":\"DeltaProcessingNotebook\",\"PurviewAccountName\":\"arkstgpurads\",\"DefaultSparkPoolName\":\"Dummy\"}},\"Source\":{\"System\":{\"SystemId\":-8,\"SystemServer\":\"https://arkstgdlsadsikjnadsl.dfs.core.windows.net\",\"AuthenticationType\":\"MSI\",\"Type\":\"ADLS\",\"Username\":null,\"Container\":\"datalakelanding\"},\"Instance\":{\"SourceRelativePath\":\"{y}/{m}/{d}/\",\"TargetRelativePath\":\"{y}/{m}/{d}/\"},\"DataFileName\":\"TestHubSpot\",\"RelativePath\":\"{y}/{m}/{d}/\",\"SchemaFileName\":\"\",\"Type\":\"Notebook-Optional\",\"WriteSchemaToPurview\":\"Disabled\"},\"Target\":{\"System\":{\"SystemId\":-8,\"SystemServer\":\"https://arkstgdlsadsikjnadsl.dfs.core.windows.net\",\"AuthenticationType\":\"MSI\",\"Type\":\"ADLS\",\"Username\":null,\"Container\":\"datalakelanding\"},\"Instance\":{\"SourceRelativePath\":\"{y}/{m}/{d}/\",\"TargetRelativePath\":\"{y}/{m}/{d}/\"},\"DataFileName\":\"HubSpotParquet\",\"RelativePath\":\"{y}/{m}/{d}/\",\"SchemaFileName\":\"\",\"Type\":\"Notebook-Optional\",\"WriteSchemaToPurview\":\"Disabled\"},\"TMOptionals\":{\"CustomDefinitions\":\"\",\"ExecuteNotebook\":\"CustomHubSpot-JsonToParquet\",\"Purview\":\"Disabled\",\"QualifiedIDAssociation\":\"TaskMasterId\",\"UseNotebookActivity\":\"Enabled\"}}"
TaskObjectJson = json.loads(TaskObject)
now = datetime.now()


#SOURCE
Source = TaskObjectJson['Source']['System']['Container'] + "@" + TaskObjectJson['Source']['System']['SystemServer'].replace("https://","") + "/"
Source = Source + TaskObjectJson['Source']['Instance']['SourceRelativePath'] + "/" + TaskObjectJson['Source']['DataFileName']
Source = Source.replace('//', '/')
Source = "abfss://" + Source
Source = Source.replace("{yyyy}", "%Y")
Source = Source.replace("{MM}", "%m")
Source = Source.replace("{dd}", "%d")
Source = Source.replace("{hh}", "%H")
Source = Source.replace("{mm}", "%M")
Source = now.strftime(Source)
print("SOURCE: " + Source)



#TARGET
Target = TaskObjectJson['Target']['System']['Container'] + "@" + TaskObjectJson['Target']['System']['SystemServer'].replace("https://","") + "/"
Target = Target + TaskObjectJson['Target']['Instance']['TargetRelativePath'] + "/" + TaskObjectJson['Target']['DataFileName']
Target = Target.replace("{yyyy}", "%Y")
Target = Target.replace("{MM}", "%m")
Target = Target.replace("{dd}", "%d")
Target = Target.replace("{hh}", "%H")
Target = Target.replace("{mm}", "%M")
Target = now.strftime(Target)

TargetDelta = Target +"/DeltaTable"
TargetDelta = TargetDelta.replace('//', '/')
TargetDelta = "abfss://" + TargetDelta
Target = Target + "/LatestVersion"
Target = Target.replace('//', '/')
Target = "abfss://" + Target
print("SAVE TARGET: " + Target)
print("DELTA SAVE TARGET: " + TargetDelta ) 

sourceDF = spark.read.option("multiline","true").json(Source)

pdf = sourceDF.toPandas()
t = pdf.iat[0,0]
fullList = []

for x in t:
    d = x.asDict() 
    fullList.append(d)

pdf = pd.DataFrame.from_records(fullList)
df=spark.createDataFrame(pdf) 
df = df.select("id","firstname","lastname","email","phone","company","createdAt","updatedAt")
#df.show()
#Convert processed dataflow to delta
sql = 'describe detail "' + TargetDelta + '"'
mergeCondition = "oldData.id" + " = newData.id"

try:
    if (spark.sql(sql).collect()[0].asDict()['format'] == 'delta'):
        print("Table already exists. Performing Merge")
        #olddt = spark.read.format('delta').load(TargetDelta)
        olddt = DeltaTable.forPath(spark, TargetDelta) 
        olddt.alias("oldData").merge(df.alias("newData"),mergeCondition).whenMatchedUpdateAll().whenNotMatchedInsertAll().execute()

        dfd = spark.read.format("delta").load(TargetDelta)
        dfd.write.format("parquet").mode("overwrite").save(Target) 

    else:
        print("Table does not exist. No error, creating new Delta Table.")    
        df.write.format("delta").save(TargetDelta)
        df.write.format("parquet").mode("overwrite").save(Target) 

except:
    print("Table does not exist. Creating new Delta Table.")    
    df.write.format("delta").save(TargetDelta)
    df.write.format("parquet").mode("overwrite").save(Target) 


